<a href="https://colab.research.google.com/github/ambideXtrous9/Langchain-Document-Chat-Assistant-using-RAG-and-Mistral7b/blob/main/Langchain_Document_Chat_Assistant_using_RAG_and_Mistral7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet langchain
!pip install --quiet bitsandbytes
!pip install --quiet chromadb==0.4.15
!pip install --quiet pypdf
!pip install --quiet accelerate
!pip install --quiet sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.6/274.6 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 723.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [31]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

In [3]:

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [4]:
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

In [5]:
model_config = transformers.AutoConfig.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

In [6]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [7]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto')

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [11]:
file = '/content/SUSHOVAN_SAHA_CV.pdf'

In [12]:
from langchain.document_loaders import PyPDFLoader

In [35]:

loader = PyPDFLoader(file)

doc = loader.load_and_split()


In [36]:
doc

[Document(page_content='Sushovan Saha\n♂phone+91 7001505741 /envel⌢pesushovansaha95@gmail.com /linkedinLinkedIn /githubGithub/kaggleKaggle\nEducation\nIndian Institute of Technology, Guwahati 2021 - 2023\nM.Tech in Data Science 8.66/10 CGPA\nIndian Institute of Engineering Science and Technology (IIEST), Shibpur 2015 - 2019\nB.Tech in Computer Science & Technology 8.01/10 CGPA\nRamakrishna Mission Vidyapith, Purulia 2013 - 2015\nBoard - WBCHSE 90 Percent\nRamharipur Ramakrishna Mission High School 2008 - 2013\nBoard - WBBSE 94.86 Percent\nWork Experience\nTVS Motor Company July 2023 – Current\nResearch & Development Engineer Bengaluru\n•Hackathon : Implemented minimal functionalities on VCU in AUTOSAR Framework to run the EV scooter iQUBE.Projects\n1. MTP : News Article based Question-Answering System Jun’22 - Jun’23\nThesis : Dr. Ashish Anand, Dept. of CSE and Dr. Prithwijit Guha, Dept. of EEE, IIT Guwahati GitHub\n•Built a QA model by fine-tuning FlanT5 onNewsQA Dataset generated fro

In [37]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [38]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,
                                               chunk_overlap=150)

In [39]:
#Create a split of the document using the text splitter
splits = text_splitter.split_documents(doc)

In [40]:
splits

[Document(page_content='Sushovan Saha\n♂phone+91 7001505741 /envel⌢pesushovansaha95@gmail.com /linkedinLinkedIn /githubGithub/kaggleKaggle\nEducation\nIndian Institute of Technology, Guwahati 2021 - 2023\nM.Tech in Data Science 8.66/10 CGPA\nIndian Institute of Engineering Science and Technology (IIEST), Shibpur 2015 - 2019\nB.Tech in Computer Science & Technology 8.01/10 CGPA\nRamakrishna Mission Vidyapith, Purulia 2013 - 2015\nBoard - WBCHSE 90 Percent\nRamharipur Ramakrishna Mission High School 2008 - 2013\nBoard - WBBSE 94.86 Percent\nWork Experience\nTVS Motor Company July 2023 – Current\nResearch & Development Engineer Bengaluru\n•Hackathon : Implemented minimal functionalities on VCU in AUTOSAR Framework to run the EV scooter iQUBE.Projects\n1. MTP : News Article based Question-Answering System Jun’22 - Jun’23\nThesis : Dr. Ashish Anand, Dept. of CSE and Dr. Prithwijit Guha, Dept. of EEE, IIT Guwahati GitHub\n•Built a QA model by fine-tuning FlanT5 onNewsQA Dataset generated fro

In [41]:
model_name = "hkunlp/instructor-large"
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(model_name=model_name,
                                   model_kwargs=model_kwargs)

In [42]:
vectordb = Chroma.from_documents(documents=splits,
                                 embedding=embeddings,
                                 persist_directory="chroma_db")

In [43]:
from transformers import AutoTokenizer, pipeline, logging, TextStreamer


In [53]:
streamer = TextStreamer(tokenizer,
                        skip_prompt = True,
                        skip_special_tokens = True)


text_pipeline = pipeline(task = 'text-generation',
                         model = model,
                         tokenizer = tokenizer,
                         temperature=0.5,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
                         top_p=0.25,  # select from top tokens whose probability add up to 15%
                         top_k=0,  # select from top 0 tokens (because zero, relies on top_p)
                         max_new_tokens=200,  # mex number of tokens to generate in the output
                         repetition_penalty=1.1,  # without this output begins repeating
                         streamer = streamer)


llm = HuggingFacePipeline(pipeline = text_pipeline)

In [54]:
from langchain.prompts import PromptTemplate

In [68]:
template = """
    Use following piece of context to answer the question in less than 30 words.

    Context : {context}

    Question : {question}

    Answer : """

In [56]:
prompt = PromptTemplate(template=template,
                        input_variables=["context", "question"])

## **using Stuff**

In [88]:
chain_type_kwargs = {"prompt": prompt}

In [87]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever(search_kwargs={"k": 5}),
    return_source_documents = True,
    chain_type_kwargs=chain_type_kwargs,
)

In [59]:
result = qa_chain("give all the projects name?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1. MTP : News Article based Question-Answering System
   2. Multi Approach Similarity Based Article Bias Prediction using LSTM and BERT
   3. Brand Logo Classification using Transfer Learning on Flickr27 Dataset
   4. Finetune Gemma2B Instruct for QA Task
   5. Rider Driven Cancellation Prediction


In [60]:
result = qa_chain("In which companies with what jobtitle did candidate work?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1. TVS Motor Company - Research & Development Engineer.


## **using refine**

In [89]:
chain_type_kwargs = {"refine_prompt": prompt}

In [90]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",
    retriever=vectordb.as_retriever(),
    return_source_documents = True,
    chain_type_kwargs=chain_type_kwargs,
)

In [91]:
result = qa_chain({"query":"give all the projects name?"})

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Answer:
1. MTP : News Article based Question-Answering System
2. Multi Approach Similarity Based Article Bias Prediction using LSTM and BERT


ValueError: Missing some input keys: {'context'}

In [74]:
result = qa_chain("In which companies with what jobtitle did candidate work?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



The candidate worked at TVS Motor Company as a Research & Development Engineer.


ValueError: Missing some input keys: {'context'}